In [ ]:
%load_ext autoreload
%autoreload 2

import os
from pathlib import Path
import shutil
from dotenv import load_dotenv

from collab_data.file_utils import expand_path, get_project_root
from collab_data.gcs_utils import GCSClient

### Grab API keys

In [ ]:
# Load environment variables from .env file
load_dotenv()

gerbils_key = os.environ.get("COLLAB_GERBILS_KEY")
data_key = os.environ.get("COLLAB_DATA_KEY")

print(f"Gerbils key: {gerbils_key}")
print(f"Data key: {data_key}")

### Grab current project data

In [ ]:
CURRENT_PROJECT = "COLLAB_DATA"
PROJECT_KEY = Path(os.environ.get(f"{CURRENT_PROJECT}_KEY"))
PROJECT_ID = "-".join(PROJECT_KEY.stem.split("-")[:-1])

# Connect to GCS
gcs_client = GCSClient(
    project_id=PROJECT_ID,
    credentials_path=expand_path(PROJECT_KEY.as_posix(), get_project_root()),
)

### Find current folders

In [ ]:
all_buckets = gcs_client.list_buckets()
print(f"Available buckets: {all_buckets}")

### Start with the curated

In [ ]:
curated_data_dir = 'fieldwork_processed'

# Find all files in the curated fieldwork
curated_files = gcs_client.glob(f"{curated_data_dir}/*")
curated_dir = curated_files[1]


# file_path = gcs_client.glob(f"{all_buckets[-1]}/field/*final*")[0]

# # Create a local path for downloading
# local_data_path = Path("./data") / Path(file_path).name

# # Download the file
# gcs_client.gcs.get_file(
#     rpath=file_path,
#     lpath=local_data_path.as_posix(),
# )

#### Create a directory and push to gcloud

In [ ]:
# gcs_prefix = f"{BUCKET_NAME}/{SESSION_DIR.name}"
session_dir = "/workspace/fieldwork-data/birds/2024-05-18/"

for root, _, files in os.walk(session_dir):
    for file in files:
        local_path = Path(root) / file
        relative_path = local_path.relative_to(session_dir)
        gcs_path = f"{curated_dir}/{relative_path.as_posix()}"
        # print (relative_path)
        # full_path = session_dir + relative_path
        gcs_client.upload_file(str(local_path), gcs_path)

# # 🔍 List contents to verify
# print("\nUploaded GCS paths:\n")
# for path in gcs_client.glob(f"{curated_dir}/**"):
#     print(path)

### Redownload colmap files

In [ ]:
colmap_fn = curated_dir + "environment/C0043/preproc/colmap/features.h5"
out_fn = "/workspace/fieldwork-data/birds/2024-02-06/environment/C0043/preproc/colmap/features.h5"

gcs_client.gcs.get_file(colmap_fn, out_fn)

In [ ]:

# file_path = gcs_client.glob(f"{all_buckets[-1]}/field/*final*")[0]